# The Genetic Algorithm

> Biomimicry is the emulation of the models, systems, and elements of nature for the purpose of solving complex human problems. The genetic algorithm, artificial neural networks, and the ant colony algorithm are some examples, to name a few.

[The world is poorly designed. But copying nature helps.](https://www.youtube.com/watch?v=iMtXqTmfta0)

For my first assignment for [Business Analytics](https://github.com/pilsung-kang/Business-Analytics-IME654-), I implemented the genetic algorithm using numpy as the main tensor calculator library, and sklearn for calculating fitness scores. The full Class can be found in `genetic_algorithm/GeneticAlgorithm.py`.

### 0. Data Loading
For this implementation of the genetic algorithm, I used a very famous toy dataset - [The Red Wine Quality Dataset.](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)


In [31]:
# %%
import numpy as np
import pandas as pd
import copy
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tqdm.notebook import tqdm
from GeneticAlgorithm import GeneticAlgorithm

GA = GeneticAlgorithm('winequality-red.csv', pop_size=50, num_generations=20, mutation_rate=0.01, crossover_rate=0.5)

In [32]:
class GeneticAlgorithm_Jupyter:
    def __init__(self, df, pop_size=10, num_generations=10, mutation_rate=0.01, crossover_rate=0.5):
        self.df = pd.read_csv(df)
        bins = (2, 5.5, 8)
        group_names = ['bad', 'good']
        target_var = 'quality'
        self.df[target_var] = pd.cut(self.df[target_var], bins = bins, labels = group_names)
        label_quality = LabelEncoder()
        self.df[target_var] = label_quality.fit_transform(self.df[target_var])
        self.data = self.df.drop(target_var, axis=1)
        self.target = self.df[target_var]

GA.df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


It consists of 11 variables that describe the target variable, being the quality of wine, ranging from scores 3 through 8. To change the given task to binary classification, I used the `pd.cut()` method to split wines with `quality` over 5.5 into `1`, and under into `0`.

### 1. Initialization

The genetic algorithm consists of multiple hyperparameters, including the number of chromosomes (population), the fitness function, crossover mechanism, and the mutation rate.

For this example, I set the hyperparameters as following:

In [33]:
GA = GeneticAlgorithm('winequality-red.csv', pop_size=50, num_generations=20, mutation_rate=0.01, crossover_rate=0.5)
print(GA)

GeneticAlgorithm(pop_size=50, num_generations=20, mutation_rate=0.01, crossover_rate=0.5)


The first step of GA is population initialization.

It refers to how many chromosomes are to be processed in one training iteration. For this example, I used a population size of 50, which in return accounts for 50 boolean arrays with 11 values, corresponding to the number of variables in the dataset.

In [34]:
def initialization(self):
    for i in range(self.pop_size):
        self.population.append(np.random.randint(2, size=len(self.var_names)).astype(bool))
    return self.population

GA.initialization()

[array([ True,  True, False,  True,  True, False, False, False,  True,
         True, False]),
 array([False,  True, False,  True,  True,  True, False,  True,  True,
        False,  True]),
 array([ True, False,  True, False,  True, False, False, False, False,
        False,  True]),
 array([False,  True, False,  True, False, False, False,  True,  True,
         True,  True]),
 array([False, False, False, False, False, False, False,  True,  True,
        False,  True]),
 array([False,  True, False,  True, False,  True,  True,  True, False,
        False,  True]),
 array([ True, False,  True,  True,  True, False,  True, False, False,
         True,  True]),
 array([False,  True, False, False,  True,  True, False, False, False,
        False, False]),
 array([False,  True, False, False, False, False, False, False,  True,
        False, False]),
 array([ True, False,  True, False,  True,  True,  True, False,  True,
         True,  True]),
 array([ True, False, False,  True, False, False, 

In [35]:
np.random.randint(2, size=len(GA.var_names)).astype(bool)

array([ True, False,  True, False, False, False, False, False,  True,
        True, False])

### 2. Fitness Evaluation

Once the population is initialized, fitness evaluation is performed for all chromosomes in the population.

For this example, I used a standard Random Forest classifier and its accuracy score.

In [36]:
def fitness_evaluation(self):
    if self.population:
        pass
    else:
        print('Initializing the first population..')
        self.population = self.initialization()
    
    acc_score = []
    for mask in tqdm(self.population, desc='Calculating Fitness Score..'):
        train_data = self.data[np.array(self.var_names)[mask]]
        x_train, x_test, y_train, y_test = train_test_split(train_data, self.target, test_size=0.2, random_state=0)
        sc = StandardScaler()
        x_train = sc.fit_transform(x_train)
        x_test = sc.fit_transform(x_test)
        rfc = RandomForestClassifier(n_estimators=200)
        rfc.fit(x_train, y_train)
        pred_rfc = rfc.predict(x_test)
        acc = accuracy_score(y_test, pred_rfc)
        acc_score.append(acc)
    fitness_dict = {}
    count = 0
    for score in acc_score:
        fitness_dict[count] = score
        count += 1

    self.fitness_dict = fitness_dict
    self.best_chromosome.append(self.population[max(self.fitness_dict, key=self.fitness_dict.get)])
    self.best_chromosome_score.append(max(self.fitness_dict.values()))
    
    print(f'Best chromosome score: {self.best_chromosome_score[-1]}')
    return self.fitness_dict

GA.fitness_evaluation()

Calculating Fitness Score..:   0%|          | 0/50 [00:00<?, ?it/s]

Best chromosome score: 0.825


{0: 0.728125,
 1: 0.765625,
 2: 0.759375,
 3: 0.821875,
 4: 0.721875,
 5: 0.78125,
 6: 0.80625,
 7: 0.634375,
 8: 0.603125,
 9: 0.784375,
 10: 0.728125,
 11: 0.759375,
 12: 0.753125,
 13: 0.71875,
 14: 0.825,
 15: 0.775,
 16: 0.759375,
 17: 0.8,
 18: 0.525,
 19: 0.759375,
 20: 0.7125,
 21: 0.659375,
 22: 0.796875,
 23: 0.70625,
 24: 0.753125,
 25: 0.753125,
 26: 0.759375,
 27: 0.8,
 28: 0.66875,
 29: 0.7375,
 30: 0.784375,
 31: 0.753125,
 32: 0.7,
 33: 0.7625,
 34: 0.73125,
 35: 0.725,
 36: 0.76875,
 37: 0.7875,
 38: 0.8125,
 39: 0.7375,
 40: 0.803125,
 41: 0.6,
 42: 0.725,
 43: 0.778125,
 44: 0.784375,
 45: 0.778125,
 46: 0.734375,
 47: 0.79375,
 48: 0.75625,
 49: 0.74375}

Each chromosome in the population is used to mask the variables used to train the RF model.

In [37]:
mask = GA.population[0]
train_data = GA.data[np.array(GA.var_names)[mask]]
train_data

,fixed acidity,volatile acidity,residual sugar,chlorides,pH,sulphates
0,7.4,0.700,1.9,0.076,3.51,0.56
1,7.8,0.880,2.6,0.098,3.20,0.68
2,7.8,0.760,2.3,0.092,3.26,0.65
3,11.2,0.280,1.9,0.075,3.16,0.58
4,7.4,0.700,1.9,0.076,3.51,0.56
...,...,...,...,...,...,...
1594,6.2,0.600,2.0,0.090,3.45,0.58
1595,5.9,0.550,2.2,0.062,3.52,0.76
1596,6.3,0.510,2.3,0.076,3.42,0.75
1597,5.9,0.645,2.0,0.075,3.57,0.71


For the first chromosome, the RF model trains using only 6 variables.

For each generation, the accuracy from the RF model is calculated and stored in `self.fitness_dict`, and the chromosome with the highest accuracy is stored in `self.best_chromosome_score`.

In [40]:
GA.fitness_dict

{0: 0.728125,
 1: 0.765625,
 2: 0.759375,
 3: 0.821875,
 4: 0.721875,
 5: 0.78125,
 6: 0.80625,
 7: 0.634375,
 8: 0.603125,
 9: 0.784375,
 10: 0.728125,
 11: 0.759375,
 12: 0.753125,
 13: 0.71875,
 14: 0.825,
 15: 0.775,
 16: 0.759375,
 17: 0.8,
 18: 0.525,
 19: 0.759375,
 20: 0.7125,
 21: 0.659375,
 22: 0.796875,
 23: 0.70625,
 24: 0.753125,
 25: 0.753125,
 26: 0.759375,
 27: 0.8,
 28: 0.66875,
 29: 0.7375,
 30: 0.784375,
 31: 0.753125,
 32: 0.7,
 33: 0.7625,
 34: 0.73125,
 35: 0.725,
 36: 0.76875,
 37: 0.7875,
 38: 0.8125,
 39: 0.7375,
 40: 0.803125,
 41: 0.6,
 42: 0.725,
 43: 0.778125,
 44: 0.784375,
 45: 0.778125,
 46: 0.734375,
 47: 0.79375,
 48: 0.75625,
 49: 0.74375}

### 3. Selection

Once fitness evaluation has been conducted for all chromosomes in the population, a selection criterion determines which chromosome in the population is fit enough to pass on its genes (e.g. its variables) to the next generation. 